In [60]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/internship-opportunities-in-india-2025/internship-in-kolkata.csv
/kaggle/input/internship-opportunities-in-india-2025/merged_internships_dataset.csv
/kaggle/input/internship-opportunities-in-india-2025/internship-in-mumbai.csv
/kaggle/input/internship-opportunities-in-india-2025/internship-in-jaipur.csv
/kaggle/input/internship-opportunities-in-india-2025/internship-in-hyderabad.csv
/kaggle/input/internship-opportunities-in-india-2025/internship-in-delhi.csv
/kaggle/input/internship-opportunities-in-india-2025/work-from-home-internships.csv
/kaggle/input/internship-opportunities-in-india-2025/internship-in-bangalore.csv
/kaggle/input/internship-opportunities-in-india-2025/internship-in-chennai.csv
/kaggle/input/internship-opportunities-in-india-2025/internship-in-pune.csv


In [61]:
df = pd.read_csv('/kaggle/input/internship-opportunities-in-india-2025/internship-in-kolkata.csv')
df.head()

,Date Time,profile,company,Location,Start Date,Stipend,Duration,Apply by Date,Offer,Education,Skills,Perks
0,2025-09-28 10:04:23,Digital Marketing,Plan D Media,Kolkata,Immediately,"₹ 9,000 - 13,000 /month",3 Months,18 Oct' 25,Nothing,Not Specified,"Client Interaction, Client Relationship, Creat...","Certificate, Flexible, Letter Of Recommendatio..."
1,2025-09-28 10:04:25,Social Media Content Creator- Video,Pixelstreet,Kolkata,Immediately,"₹ 6,000 - 12,000 /month",3 Months,18 Oct' 25,Nothing,Not Specified,"Digital Marketing, Digital Marketing Tools, So...","Certificate, Mentorship, 5 Days A Week"
2,2025-09-28 10:04:27,Telecalling (Female),SAK GROUP,Kolkata,Immediately,"₹ 4,000 - 10,000 /month",3 Months,19 Oct' 25,Nothing,Not Specified,"Digital Marketing, English Proficiency (Spoken)","Certificate, Letter Of Recommendation"
3,2025-09-28 10:04:29,Campus Ambassador,Youth Marketer,Ahmedabad,Immediately,Not Available,3 Months,22 Oct' 25,Nothing,2. Are currently enrolled in an undergraduate ...,"Effective Communication, Interpersonal skills,...","Certificate, Flexible, Letter Of Recommendatio..."
4,2025-09-28 10:04:31,Content and Social Media Marketing,Iblix Digital,Kolkata,Immediately,"₹ 4,000 - 5,000 /month",3 Months,23 Oct' 25,Nothing,Not Specified,"Acting, Instagram Marketing, Research and Anal...","Certificate, Flexible"


In [62]:
df.columns.tolist()

['Date Time',
 'profile',
 'company',
 'Location',
 'Start Date',
 'Stipend',
 'Duration',
 'Apply by Date',
 'Offer',
 'Education',
 'Skills',
 'Perks']

In [63]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder


# ---------- 🧹 CLEANING STIPEND ----------
def extract_stipend(value):
    if pd.isnull(value):
        return np.nan
    nums = re.findall(r'\d+', str(value))
    if len(nums) == 1:
        return float(nums[0])
    elif len(nums) == 2:
        return (float(nums[0]) + float(nums[1])) / 2  # average of range
    else:
        return np.nan

df['Stipend'] = df['Stipend'].apply(extract_stipend)

# ---------- ⏳ CLEANING DURATION ----------
# Convert everything to string first, then extract digits
df['Duration'] = df['Duration'].astype(str).str.extract('(\d+)')
df['Duration'] = pd.to_numeric(df['Duration'], errors='coerce')

# ---------- 🗓️ CLEANING DATES ----------
for col in ['Date Time', 'Start Date', 'Apply by Date']:
    df[col] = pd.to_datetime(df[col], errors='coerce')

df['Month'] = df['Date Time'].dt.month
df['Year'] = df['Date Time'].dt.year

# ---------- 🔠 ENCODING TEXT COLUMNS ----------
le = LabelEncoder()
for col in ['profile', 'company', 'Location', 'Education']:
    df[col] = le.fit_transform(df[col].astype(str))

# ---------- ⚙️ EXTRA FEATURES ----------
df['Skill_Count'] = df['Skills'].apply(lambda x: len(str(x).split(',')) if pd.notnull(x) else 0)
df['Has_Perks'] = df['Perks'].apply(lambda x: 1 if pd.notnull(x) else 0)

# ---------- 🧽 CLEAN FINAL DATA ----------
df.dropna(subset=['Stipend'], inplace=True)  # remove unpaid/missing stipend

df.head()

/tmp/ipykernel_99/1910434767.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce')
/tmp/ipykernel_99/1910434767.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce')


,Date Time,profile,company,Location,Start Date,Stipend,Duration,Apply by Date,Offer,Education,Skills,Perks,Month,Year,Skill_Count,Has_Perks
8,2025-09-28 10:04:39,18,37,5,NaT,4.0,6.0,2025-10-25,Nothing,28,"E-commerce, Effective Communication, MS-Excel,...","Certificate, Flexible, Letter Of Recommendatio...",9,2025,4,1
18,2025-09-28 10:04:59,7,54,4,NaT,1.0,2.0,2025-10-24,Nothing,28,"Client Interaction, Effective Communication, F...","Certificate, Flexible",9,2025,5,1
25,2025-09-28 10:05:15,16,90,4,NaT,2.5,3.0,2025-10-25,Nothing,2,"Ethical Hacking, Java","Certificate, Letter Of Recommendation",9,2025,2,1
27,2025-09-28 10:05:19,7,92,5,NaT,2.5,6.0,2025-10-24,Nothing,28,"English Proficiency (Spoken), English Proficie...","Certificate, Letter Of Recommendation",9,2025,7,1
28,2025-09-28 10:05:21,7,2,5,NaT,4.0,6.0,2025-10-24,Nothing,28,English Proficiency (Spoken),"Certificate, Flexible, Letter Of Recommendation",9,2025,1,1


In [64]:
df.isnull().sum()

Date Time         0
profile           0
company           0
Location          0
Start Date       22
Stipend           0
Duration          0
Apply by Date     2
Offer             0
Education         0
Skills            0
Perks             0
Month             0
Year              0
Skill_Count       0
Has_Perks         0
dtype: int64

In [65]:
df = df[['profile', 'company', 'Location', 'Duration', 'Education', 'Skills', 'Perks', 'Stipend']]
df.isnull().sum()

profile      0
company      0
Location     0
Duration     0
Education    0
Skills       0
Perks        0
Stipend      0
dtype: int64

In [66]:
X = df.drop(['Stipend'],axis=1)
y = df['Stipend']

In [67]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [68]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Find which columns are still categorical
categorical_cols = X_train.select_dtypes(include=['object']).columns

# Apply one-hot encoding
ct = ColumnTransformer(
    transformers=[('encoder', OneHotEncoder(handle_unknown='ignore'), categorical_cols)],
    remainder='passthrough'
)

X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)

In [69]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [70]:
model = Sequential()
model.add(Dense(128, activation='relu',input_shape=(X_train.shape[1],)))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(1))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [71]:
model.compile(loss='mse',optimizer='adam',metrics=['mae'])

In [72]:
model.fit(X_train,y_train,epochs = 25,validation_split=0.2)

Epoch 1/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 26.4827 - mae: 4.7498 - val_loss: 16.8215 - val_mae: 3.5261
Epoch 2/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step - loss: 8.9385 - mae: 2.4370 - val_loss: 5.8102 - val_mae: 2.4037
Epoch 3/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step - loss: 5.9851 - mae: 1.9063 - val_loss: 6.7240 - val_mae: 1.9844
Epoch 4/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step - loss: 10.1685 - mae: 2.7511 - val_loss: 7.5699 - val_mae: 1.8988
Epoch 5/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step - loss: 10.5978 - mae: 2.7996 - val_loss: 6.2106 - val_mae: 1.9710
Epoch 6/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step - loss: 7.3825 - mae: 2.3630 - val_loss: 5.0274 - val_mae: 2.0771
Epoch 7/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step - loss: 4.3272 - mae: 1.7805 - val_loss: 4.9865 - val_mae: 2.1899
Epoch 8/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step - loss: 2.6888 - mae: 1.2187 - val_loss: 5.9951 - val_mae: 2.2971
Epoch 9/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step - loss: 2.4833 - mae:

In [73]:
y_pred = model.predict(X_test)
y_pred

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step


array([[4.2449303],
       [2.4192595],
       [3.6736267],
       [3.2192411],
       [3.672446 ],
       [4.6247873],
       [5.4289174]], dtype=float32)

In [74]:
from sklearn.metrics import mean_absolute_error
print(f"Error:\t {mean_absolute_error(y_pred,y_test)}")

Error:	 1.1928889410836356
